In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/team1_export")


True

In [0]:
dfs['silver.silver_flight_info'].write.mode('overwrite').option('header', 'true').csv('/dbfs/FileStore/team1_export/silver_silver_flight_info.csv')


# 간단한 정리
### 혼잡도 분류 모델을 위한 데이터 소스 및 테이블 정리

---

#### 1. 주요 데이터 소스 및 테이블 개요

| 구분   | 테이블명                        | 설명                                   |
|--------|---------------------------------|----------------------------------------|
| 정적   | silver_flight_info              | 항공 운행 정보 (정기운항편 상세)        |
| 정적   | bronze_flight_info              | 항공 운행 정보 (실시간 수집)            |
| 정적   | silver_temperature              | 공항 기상 관측 데이터                   |
| 실시간 | api_silver_departure_forecast   | 출국장별 예상 승객수 (실시간)           |
| 실시간 | api_bronze_departure_forecast   | 출국장별 예상 승객수 (브론즈)           |
| 실시간 | api_bronze_area_weather_data    | 실시간 항공 기상 정보                   |
| 실시간 | api_bronze_departure_weather    | 실시간 출발 항공편 도착지 기상 정보     |
| 실시간 | api_silver_parkinglot           | 실시간 인천공항 주차 현황 정보          |
| 실시간 | api_bronze_indoorair_quality    | 실시간 인천공항 여객터미널 실내 대기질 정보 |
| 정적   | silver_monthly                  | 인천공항 월별 통계                      |
| 정적   | silver_hourly                   | 인천공항 시간대별 통계                  |
| 정적   | silver_weekday                  | 요일별 항공 통계                        |
| 정적   | silver_airline                  | 항공사별 항공 데이터                    |
| 정적   | bronze_departure_forecast_t1    | T1 출국장 예상 승객수 (브론즈)           |
| 정적   | bronze_departure_forecast_t2    | T2 출국장 예상 승객수 (브론즈)           |
| 정적   | bronze_temperature              | 공항 기상 관측 데이터 (브론즈)           |

---

#### 2. 혼잡도 분류에 활용 가능한 주요 변수 예시

- **출국장별 예상 승객수**
  - 실시간: `api_silver_departure_forecast`, `api_bronze_departure_forecast`
  - 정적: `bronze_departure_forecast_t1`, `bronze_departure_forecast_t2`

- **공항 기상 데이터**
  - 실시간: `api_bronze_area_weather_data`
  - 정적: `silver_temperature`, `bronze_temperature`

- **항공 운항 정보**
  - 정적: `silver_flight_info`, `bronze_flight_info`
  - 시간대별/월별/요일별: `silver_hourly`, `silver_monthly`, `silver_weekday`, `silver_airline`

- **기타 실시간 환경 변수**
  - 주차장 혼잡: `api_silver_parkinglot`
  - 실내 대기질: `api_bronze_indoorair_quality`

---

#### 3. 데이터 활용 및 모델 학습 흐름 예시

1. **데이터 불러오기**
    - 위 테이블에서 혼잡도 분류에 필요한 변수(예상 승객수, 시간대, 터미널, 기상 등) 선택

2. **피처 엔지니어링**
    - 시간대별, 터미널별, 출국장별 데이터 통합
    - 기상, 환경, 운항정보 등 외부 변수 조인

3. **타겟 정의**
    - 예: 시간대별 출국장 혼잡도(여유/보통/혼잡 등급)

4. **모델 학습 및 평가**
    - 분류 모델(RandomForest, XGBoost 등) 적용
    - 혼잡도 등급 예측 정확도 평가



## 1. 레퍼런스 기반 접근

- "딥러닝 기반 항공기 지연 예측 분석" 논문을 주요 참고자료로 삼아,
- 실제 지연 시간(분 단위)을 예측하는 **회귀 문제**로 접근
- 논문에서처럼 운항 정보, 기상 데이터 등 다양한 변수를 활용해 모델 설계

---

### 2. 모델링 전략

- 여러 모델을 비교:
  - 딥러닝(FCNN 등)
  - 랜덤포레스트
  - XGBoost
  - 결정트리
  - 선형회귀 등
- 각 모델의 성능을 **평가지표(MAE, RMSE 등)**로 비교
- 가장 예측력이 좋은 모델을 최종 선택

---

### 3. 실무 적용 방식

- 논문 방식처럼 결측치 처리, 변수 인코딩, 데이터 불균형 보정 등 데이터 전처리 꼼꼼히 진행
- 학습/검증/테스트셋 분리로 과적합 방지 및 일관된 성능 평가
- 모델별 예측 결과를 정량적으로 비교하여 실제 서비스에 적용할 모델 선정

---

### 4. 참고 표: 모델 비교 예시

| 모델         | MAE(분) | RMSE(분) | 특징               |
|--------------|---------|----------|--------------------|
| 딥러닝       | 14.5    | 20.1     | 변수 복합성에 강함 |
| 랜덤포레스트 | 17.0    | 23.3     | 해석력/속도 우수   |
| XGBoost      | 18.8    | 25.0     | 변수 중요도 해석   |
| 선형회귀     | 19.1    | 26.2     | 단순/빠름          |


# 항공기 지연 예측(회귀) 관련 정리
사용할 테이블
1. 골드테이블(gold_flight_info) : 
날짜, 연/월/일, 요일, 공휴일 여부
계획/실제 출발시간, 항공사, 편명, 도착지, 항공종류
추정 여객/화물, 혼잡도, 운항 상태
지연시간(타겟, delayed_time)
지연범주, 지연이유
풍속, 가시거리, 일기현상, 기온, 이슬점온도, 강수량 등

2. 실버텡이블(보조 및 확장)
silver_flight_info,temperature  이게 골드에 들어간건가?
holiday는 

In [0]:
df = spark.table("`1team-postgresql-connection_catalog`.`gold`.`gold_flight_info`")


In [0]:

cols = [
    "destination_city",
    "scheduled_hour",
    "temperature",
    "wind_speed",
    "visibillity",
    "weather_phenomenon",
    "congestion_level",
    "final_est_passenger",
    "delayed_time"
]

df = spark.table("`1team-postgresql-connection_catalog`.`gold`.`gold_flight_info`").select(cols)


df.printSchema()
df.show(5)


root
 |-- destination_city: string (nullable = true)
 |-- scheduled_hour: integer (nullable = true)
 |-- temperature: string (nullable = true)
 |-- wind_speed: integer (nullable = true)
 |-- visibillity: integer (nullable = true)
 |-- weather_phenomenon: integer (nullable = true)
 |-- congestion_level: string (nullable = true)
 |-- final_est_passenger: float (nullable = true)
 |-- delayed_time: integer (nullable = true)

+----------------+--------------+-----------+----------+-----------+------------------+----------------+-------------------+------------+
|destination_city|scheduled_hour|temperature|wind_speed|visibillity|weather_phenomenon|congestion_level|final_est_passenger|delayed_time|
+----------------+--------------+-----------+----------+-----------+------------------+----------------+-------------------+------------+
|     도쿄/나리타|             7|      11.80|         3|       9000|                10|            보통|              25.08|          11|
|          호찌민|             7|

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/team1_export")


True

In [0]:
df.write.mode("overwrite").option("header", "true").csv("/dbfs/FileStore/team1_export/gold_flight_info_selected.csv")


In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/team1_export/gold_flight_info_selected.csv"))


---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-4805356429454380>, line 1
----> 1 display(dbutils.fs.ls("dbfs:/FileStore/team1_export/gold_flight_info_selected.csv"))

File /databricks/python_shell/lib/dbruntime/dbutils.py:158, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    156 exc.__context__ = None
    157 exc.__cause__ = None
--> 158 raise exc

ExecutionError: An error occurred while calling o463.ls.
: java.io.FileNotFoundException: No such file or directory dbfs:/FileStore/team1_export/gold_flight_info_selected.csv
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:197)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:66)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.

In [0]:
df.write.mode("overwrite").option("header", "true").csv("/dbfs/FileStore/team1_export/gold_flight_info_selected.csv")


In [0]:
df.write.mode("overwrite").option("header", "true").csv("/dbfs/FileStore/team1_export/gold_flight_info_selected.csv")


In [0]:
display(dbutils.fs.ls("dbfs:/FileStore"))


path,name,size,modificationTime
dbfs:/FileStore/team1_export/,team1_export/,0,1752484261000


In [0]:
df.write.mode("overwrite").option("header", "true").csv("/dbfs/FileStore/team1_export/gold_flight_info_selected.csv")


In [0]:
df.show(5)


+----------------+--------------+-----------+----------+-----------+------------------+----------------+-------------------+------------+
|destination_city|scheduled_hour|temperature|wind_speed|visibillity|weather_phenomenon|congestion_level|final_est_passenger|delayed_time|
+----------------+--------------+-----------+----------+-----------+------------------+----------------+-------------------+------------+
|     도쿄/나리타|             7|      11.80|         3|       9000|                10|            보통|              25.08|          11|
|          호찌민|             7|      11.80|         3|       9000|                10|            보통|             148.02|          19|
|            다낭|             7|      11.80|         3|       9000|                10|            보통|              62.62|          20|
|     도쿄/나리타|             7|      11.80|         3|       9000|                10|            보통|              98.52|          16|
|        후쿠오카|             7|      11.80|         3|     

도대체 뭐가 문제길래 30분을 버리게 함?? llm이 문제인거야 이게 문제인거야 폴더 만들고 파일 csv로 저장인데 왜 계속 비어ㅣㅇㅆ다그러아머라ㅣㅇ러마ㅣㄴ러ㅣ
왜 ml studio에 데이터브릭스 연동은 안되나? postgres도안되고
